<a href="https://colab.research.google.com/github/Ngga24/BarcodeScanner/blob/main/2318120DataCleasing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install API Kaggle

In [ ]:
# Install Kaggle API
!pip install kaggle
from google.colab import files

# Upload kaggle.json (bisa download dari: Kaggle > My Account > API > Create New API Token)
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle (2).json


# Donwload Dataset Dari Kaggle

In [ ]:
# Download dataset
!kaggle datasets download -d ahmedmohamed2003/retail-store-sales-dirty-for-data-cleaning -p /content/

# Unzip file
!unzip /content/retail-store-sales-dirty-for-data-cleaning.zip -d /content/

Dataset URL: https://www.kaggle.com/datasets/ahmedmohamed2003/retail-store-sales-dirty-for-data-cleaning
License(s): CC-BY-SA-4.0
retail-store-sales-dirty-for-data-cleaning.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  /content/retail-store-sales-dirty-for-data-cleaning.zip
  inflating: /content/retail_store_sales.csv  


# Load Dataset Ke Pandas

In [ ]:
import pandas as pd

# Sesuaikan nama file CSV hasil unzip
df = pd.read_csv("/content/retail_store_sales.csv")

print("Shape awal:", df.shape)
df.head()

Shape awal: (12575, 11)


,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied
0,TXN_6867343,CUST_09,Patisserie,Item_10_PAT,18.5,10.0,185.0,Digital Wallet,Online,2024-04-08,True
1,TXN_3731986,CUST_22,Milk Products,Item_17_MILK,29.0,9.0,261.0,Digital Wallet,Online,2023-07-23,True
2,TXN_9303719,CUST_02,Butchers,Item_12_BUT,21.5,2.0,43.0,Credit Card,Online,2022-10-05,False
3,TXN_9458126,CUST_06,Beverages,Item_16_BEV,27.5,9.0,247.5,Credit Card,Online,2022-05-07,NaN
4,TXN_4575373,CUST_05,Food,Item_6_FOOD,12.5,7.0,87.5,Digital Wallet,Online,2022-10-02,False


# Menangani Missing Value

In [ ]:
df['Item'] = df['Item'].fillna(df['Category'].astype(str) + "_Unknown")
df['Price Per Unit'] = df['Price Per Unit'].fillna(df['Price Per Unit'].median())
df['Quantity'] = df['Quantity'].fillna(df['Quantity'].median())
df['Total Spent'] = df['Price Per Unit'] * df['Quantity']
df['Discount Applied'] = df['Discount Applied'].fillna(False)

/tmp/ipython-input-411788409.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Discount Applied'] = df['Discount Applied'].fillna(False)


# Standarisasi Kategorikal

In [ ]:
df['Payment Method'] = df['Payment Method'].str.strip().str.title()
df['Category'] = df['Category'].str.strip().str.title()
df['Location'] = df['Location'].str.strip().str.title()
valid_payment = ['Cash', 'Credit Card', 'Digital Wallet']
df = df[df['Payment Method'].isin(valid_payment)]

# Format & Validasi Tanggal

In [ ]:
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'], errors='coerce')
df = df.dropna(subset=['Transaction Date'])
df['Year'] = df['Transaction Date'].dt.year
df['Month'] = df['Transaction Date'].dt.month
df['Day'] = df['Transaction Date'].dt.day

# Buang nilai tidak logis & outlier ekstrem

In [ ]:
df = df[(df['Quantity'] > 0) & (df['Price Per Unit'] > 0)]
df = df[(np.abs((df['Price Per Unit'] - df['Price Per Unit'].mean()) / df['Price Per Unit'].std()) < 3)]
df = df[(np.abs((df['Quantity'] - df['Quantity'].mean()) / df['Quantity'].std()) < 3)]

# Hapus duplikat berdasarkan Transaction ID

In [ ]:
df = df.drop_duplicates(subset=['Transaction ID'], keep='first')

# Validasi cross-field

In [ ]:
mask = (df['Payment Method'] == "Cash") & (df['Location'] != "In-Store")
df.loc[mask, 'Location'] = "In-Store"
mask = (df['Location'] == "Online") & (df['Payment Method'] == "Cash")
df = df[~mask]

print("Shape akhir:", df.shape)
df.head(20)

Shape akhir: (12575, 14)


,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied,Year,Month,Day
0,TXN_6867343,CUST_09,Patisserie,Item_10_PAT,18.5,10.0,185.0,Digital Wallet,Online,2024-04-08,True,2024,4,8
1,TXN_3731986,CUST_22,Milk Products,Item_17_MILK,29.0,9.0,261.0,Digital Wallet,Online,2023-07-23,True,2023,7,23
2,TXN_9303719,CUST_02,Butchers,Item_12_BUT,21.5,2.0,43.0,Credit Card,Online,2022-10-05,False,2022,10,5
3,TXN_9458126,CUST_06,Beverages,Item_16_BEV,27.5,9.0,247.5,Credit Card,Online,2022-05-07,False,2022,5,7
4,TXN_4575373,CUST_05,Food,Item_6_FOOD,12.5,7.0,87.5,Digital Wallet,Online,2022-10-02,False,2022,10,2
5,TXN_7482416,CUST_09,Patisserie,Patisserie_Unknown,23.0,10.0,230.0,Credit Card,Online,2023-11-30,False,2023,11,30
6,TXN_3652209,CUST_07,Food,Item_1_FOOD,5.0,8.0,40.0,Credit Card,In-Store,2023-06-10,True,2023,6,10
7,TXN_1372952,CUST_21,Furniture,Furniture_Unknown,33.5,6.0,201.0,Digital Wallet,In-Store,2024-04-02,True,2024,4,2
8,TXN_9728486,CUST_23,Furniture,Item_16_FUR,27.5,1.0,27.5,Credit Card,In-Store,2023-04-26,False,2023,4,26
9,TXN_2722661,CUST_25,Butchers,Item_22_BUT,36.5,3.0,109.5,Cash,In-Store,2024-03-14,False,2024,3,14


# Simpan Dataset Bersih

In [ ]:
df.to_csv("/content/retail_store_sales_cleaned.csv", index=False)
print("Dataset bersih disimpan di: /content/retail_store_sales_cleaned.csv")

Dataset bersih disimpan di: /content/retail_store_sales_cleaned.csv
